In [79]:
import os
import json
import pandas as pd
import time
from tqdm import tqdm
import gc

path_data = r'C:\Users\satme\Downloads\spotify_million_playlist_dataset'
path_df_data = r'C:\Users\satme\Downloads\spotify_million_playlist_dataset\df_data'

In [49]:
filenames = os.listdir(path_data)

In [53]:
playlist_col = ['collaborative', 'duration_ms', 'modified_at', 
                'name', 'num_albums', 'num_artists', 'num_edits',
                'num_followers', 'num_tracks', 'pid']
tracks_col = ['album_name', 'album_uri', 'artist_name', 'artist_uri', 
                  'duration_ms', 'track_name', 'track_uri'] 
playlist_test_col = ['name', 'num_holdouts', 'num_samples', 'num_tracks', 'pid']
    
filenames = os.listdir(path_data)
    
data_playlists = []
data_tracks = []
playlists = []
tracks = set()
    
total_time = 0

print("Reading the dataset")
for filename in tqdm(filenames):
    start_time = time.time()  
    fullpath = os.sep.join((path_data, filename))
    f = open(fullpath)
    js = f.read()
    f.close()
    
    mpd_slice = json.loads(js)
    for playlist in mpd_slice['playlists']:
            data_playlists.append([playlist[col] for col in playlist_col])
            for track in playlist['tracks']:
                playlists.append([playlist['pid'], track['track_uri'], track['pos']])
                if track['track_uri'] not in tracks:
                    data_tracks.append([track[col] for col in tracks_col])
                    tracks.add(track['track_uri'])
    duration = time.time() - start_time
    total_time += duration
    #print("Time elapsed: ",duration)
print("Total time elapsed: ",total_time)
gc.collect()
print("Reading the challenge dataset")
f = open(r'C:\Users\satme\Downloads\challenge_set.json\challenge_set.json')
js = f.read()
f.close()
mpd_slice = json.loads(js)
data_playlists_test = []
playlists_test = []

Reading the dataset


100%|██████████| 20/20 [00:06<00:00,  2.89it/s]


Total time elapsed:  6.911773443222046
Reading the challenge dataset


In [55]:
for playlist in tqdm(mpd_slice['playlists']):
        data_playlists_test.append([playlist.get(col, '') for col in playlist_test_col])
        for track in playlist['tracks']:
            playlists_test.append([playlist['pid'], track['track_uri'], track['pos']])
            if track['track_uri'] not in tracks:
                data_tracks.append([track[col] for col in tracks_col])
                tracks.add(track['track_uri'])

100%|██████████| 10000/10000 [00:00<00:00, 42812.87it/s]


In [57]:
df_playlists_info = pd.DataFrame(data_playlists, columns=playlist_col)
df_playlists_info['collaborative'] = df_playlists_info['collaborative'].map({'false': False, 'true': True})

In [58]:
df_tracks = pd.DataFrame(data_tracks, columns=tracks_col)
df_tracks['tid'] = df_tracks.index

In [61]:
track_uri2tid = df_tracks.set_index('track_uri').tid
#df_playlists = pd.DataFrame.from_records(playlists, columns=['pid', 'tid', 'pos'])
df_playlists = pd.DataFrame.from_records(playlists, columns=['pid', 'tid', 'pos'])
df_playlists.tid = df_playlists.tid.map(track_uri2tid)

In [93]:
df_playlists_test_info = pd.DataFrame.from_records(data_playlists_test, columns=playlist_test_col)

df_playlists_test = pd.DataFrame.from_records(playlists_test, columns=['pid', 'tid', 'pos'])
df_playlists_test.tid = df_playlists_test.tid.map(track_uri2tid)

df_tracks.to_hdf(path_data+'\df_data\df_tracks_info.hdf', key='abc')
df_playlists.to_hdf(path_data+'\df_data\df_tracks.hdf', key='abc')
df_playlists_info.to_hdf(path_data+'\df_data\df_playlists_info.hdf', key='abc')
df_playlists_test.to_hdf(path_data+'\df_data\challenge_set\df_tracks_test.hdf', key='abc')
df_playlists_test_info.to_hdf(path_data+'\df_data\challenge_set\df_playlists_test_info.hdf', key='abc')

In [94]:
if __name__ == "__main__":
    print(__doc__)
    path = path_data
    # Check existence of folder df_data
    list_dir = os.listdir(path)
    if ("df_data" not in list_dir):
        print("Create df_data folder")
        os.makedirs(path+"/df_data")
        os.makedirs(path+"/df_data/challenge_set")

# call main function      
create_df_data(path)

Automatically created module for IPython interactive environment


In [99]:
df_tracks = pd.read_hdf(path_df_data + '\df_tracks.hdf')
df_tracks_info = pd.read_hdf(path_df_data + '\df_tracks_info.hdf')
df_playlists_info = pd.read_hdf(path_df_data + '\df_playlists_info.hdf')


In [101]:
df_playlists_info.head(20)

,collaborative,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid
0,False,1434050,1487721600,Prelude,7,7,2,1,7,20000
1,False,7576808,1506211200,adam,34,30,4,1,36,20001
2,False,7035649,1495843200,WARMUP,31,28,7,1,31,20002
3,False,10219395,1499472000,Running,41,28,11,1,42,20003
4,False,4009411,1424736000,2015,17,14,7,1,18,20004
5,False,6494994,1505260800,christmas,25,22,2,3,34,20005
6,False,7744323,1487808000,Adventure,10,7,7,2,39,20006
7,False,8688708,1508371200,Country,35,31,15,1,41,20007
8,False,19256226,1481500800,sleep 2,66,55,29,1,75,20008
9,False,6074367,1478044800,mellow,23,15,11,1,26,20009


In [104]:
# Build challenge set
# Read data
df_tracks = pd.read_hdf(path_df_data + '\df_tracks.hdf')
df_unique_tracks  = pd.read_hdf(path_df_data + '\df_tracks_info.hdf')
df_playlists = pd.read_hdf(path_df_data + '\df_playlists_info.hdf')  
    

In [109]:
import pandas as pd
from tqdm import tqdm
#from helper import alertError,alertFinishJob

In [112]:
# Pandas table sorted by 
df_temp = df_tracks.copy()
df_temp['count'] = 1

In [113]:
# Ensure any track must appear in training
df_track_distr = df_temp.groupby(['tid'])['count'].sum().sort_values(ascending=False)
del df_temp
    

In [118]:
list_size = 100
criteria_list = [200,200,100,100,50,50,25,25,10,5]

pid_list = []
#pid_unfold_list=[]
df_temp = df_playlists.loc[:,['num_tracks','pid']]
df_temp = df_temp.set_index('pid')
for criteria in tqdm(criteria_list):
#criteria = 200
    df_filter = df_temp[df_temp.num_tracks > criteria]

100%|██████████| 10/10 [00:00<00:00, 882.23it/s]


In [ ]:
 # Create an empty list to contain the pid
        
list_1000 = []
while (True):
    # randomly pick 1 pid in df_filter
    ran = df_filter.sample(n=1)
    # get the value of pid
    pid = ran.index[0]
#        pid = ran.pid.values[0]


In [ ]:
# get list of tid (this code uses so many memory)
tid_arr = df_tracks[df_tracks.pid == pid].tid
# decrease frequency by 1
temp = df_track_distr[tid_arr] - 1

In [ ]:
if (temp.any() != 0):
    df_track_distr[tid_arr] = temp
    list_1000.append(pid)

    df_filter = df_filter.drop(pid)

    if (len(list_1000) == list_size):
        break